In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("라이브러리 로드 완료")


In [ ]:
# 데이터 로드
df = pd.read_csv('../data/processed/filtered_data.csv')

print(f"전체 데이터 크기: {df.shape}")
print(f"포함된 국가: {df['Country Name'].unique()}")
print(f"데이터 기간: {df.columns[4:-1].min()} - {df.columns[4:-1].max()}")


In [ ]:
# 물/위생 관련 사망률 데이터 필터링
wash_mortality = df[df['Indicator Code'] == 'SH.STA.WASH.P5'].copy()

print(f"물/위생 사망률 데이터 크기: {wash_mortality.shape}")
print("\n=== 데이터 기본 정보 ===")
print(wash_mortality[['Country Name', 'Indicator Name', 'Indicator Code']].head())


In [ ]:
# 연도별 컬럼을 행으로 변환 (melt)
year_columns = [col for col in wash_mortality.columns if col.isdigit()]
wash_long = wash_mortality.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Mortality_Rate'
)

# 데이터 타입 변환
wash_long['Year'] = wash_long['Year'].astype(int)
wash_long['Mortality_Rate'] = pd.to_numeric(wash_long['Mortality_Rate'], errors='coerce')

# 결측값이 아닌 데이터만 필터링
wash_clean = wash_long.dropna(subset=['Mortality_Rate']).copy()

print(f"변환된 데이터 크기: {wash_clean.shape}")
print(f"사용 가능한 연도 범위: {wash_clean['Year'].min()} - {wash_clean['Year'].max()}")


In [ ]:
# 기본 통계 정보
print("=== 기본 통계 정보 ===")
print(wash_clean['Mortality_Rate'].describe())

print("\n=== 국가별 통계 ===")
country_stats = wash_clean.groupby('Country Name')['Mortality_Rate'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)
print(country_stats)


In [ ]:
# 데이터 가용성 분석
print("=== 연도별 데이터 가용성 ===")
availability = wash_clean.groupby(['Country Name', 'Year']).size().reset_index(name='count')
pivot_availability = availability.pivot(index='Year', columns='Country Name', values='count').fillna(0)

print("각 국가별 연도별 데이터 존재 여부:")
print(pivot_availability.astype(bool).astype(int))

# 각 국가별 데이터 기간
print("\n=== 국가별 데이터 기간 ===")
for country in wash_clean['Country Name'].unique():
    country_data = wash_clean[wash_clean['Country Name'] == country]
    print(f"{country}: {country_data['Year'].min()} - {country_data['Year'].max()} ({len(country_data)}개 데이터포인트)")


In [ ]:
# 시계열 트렌드 시각화
plt.figure(figsize=(14, 8))

for country in wash_clean['Country Name'].unique():
    country_data = wash_clean[wash_clean['Country Name'] == country]
    plt.plot(country_data['Year'], country_data['Mortality_Rate'], 
             marker='o', linewidth=2, markersize=6, label=country)

plt.title('안전하지 않은 물/위생으로 인한 사망률 추이\n(인구 10만명당)', fontsize=16, fontweight='bold')
plt.xlabel('연도', fontsize=12)
plt.ylabel('사망률 (인구 10만명당)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 최근 5년 평균 계산
recent_years = wash_clean[wash_clean['Year'] >= wash_clean['Year'].max() - 4]
recent_avg = recent_years.groupby('Country Name')['Mortality_Rate'].mean().sort_values(ascending=False)
print("\n=== 최근 5년 평균 사망률 (높은 순) ===")
for country, rate in recent_avg.items():
    print(f"{country}: {rate:.2f}명/10만명")


In [ ]:
# 국가별 박스플롯
plt.figure(figsize=(12, 8))
sns.boxplot(data=wash_clean, x='Country Name', y='Mortality_Rate')
plt.title('국가별 물/위생 사망률 분포', fontsize=16, fontweight='bold')
plt.xlabel('국가', fontsize=12)
plt.ylabel('사망률 (인구 10만명당)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 각 국가의 최고/최저 사망률 연도
print("=== 각 국가의 최고/최저 사망률 ===")
for country in wash_clean['Country Name'].unique():
    country_data = wash_clean[wash_clean['Country Name'] == country]
    max_row = country_data.loc[country_data['Mortality_Rate'].idxmax()]
    min_row = country_data.loc[country_data['Mortality_Rate'].idxmin()]
    
    print(f"\n{country}:")
    print(f"  최고: {max_row['Mortality_Rate']:.2f}명/10만명 ({max_row['Year']}년)")
    print(f"  최저: {min_row['Mortality_Rate']:.2f}명/10만명 ({min_row['Year']}년)")
    print(f"  개선율: {((max_row['Mortality_Rate'] - min_row['Mortality_Rate']) / max_row['Mortality_Rate'] * 100):.1f}% 감소")


In [ ]:
# 연도별 변화율 분석
def calculate_change_rate(group):
    group = group.sort_values('Year')
    group['Change_Rate'] = group['Mortality_Rate'].pct_change() * 100
    return group

wash_with_change = wash_clean.groupby('Country Name').apply(calculate_change_rate).reset_index(drop=True)

# 변화율 시각화
plt.figure(figsize=(14, 8))
for country in wash_with_change['Country Name'].unique():
    country_data = wash_with_change[wash_with_change['Country Name'] == country]
    plt.plot(country_data['Year'], country_data['Change_Rate'], 
             marker='s', linewidth=2, markersize=5, label=country, alpha=0.8)

plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.title('물/위생 사망률 전년 대비 변화율', fontsize=16, fontweight='bold')
plt.xlabel('연도', fontsize=12)
plt.ylabel('변화율 (%)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 평균 연간 변화율
print("=== 평균 연간 변화율 ===")
avg_change = wash_with_change.groupby('Country Name')['Change_Rate'].mean().sort_values()
for country, rate in avg_change.items():
    direction = "개선" if rate < 0 else "악화"
    print(f"{country}: {rate:.2f}% ({direction})")


In [ ]:
# 히트맵으로 연도별 국가별 사망률 비교
pivot_data = wash_clean.pivot(index='Year', columns='Country Name', values='Mortality_Rate')

plt.figure(figsize=(10, 12))
sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='Reds', 
            cbar_kws={'label': '사망률 (인구 10만명당)'})
plt.title('연도별 국가별 물/위생 사망률 히트맵', fontsize=16, fontweight='bold')
plt.xlabel('국가', fontsize=12)
plt.ylabel('연도', fontsize=12)
plt.tight_layout()
plt.show()
